# Pretraining tail network

In [4]:
from src.dlc_practical_prologue import generate_pair_sets
from src.utils import load_class_data, load_target_data, load_all_data, print_param_count
from src.models import *
from src.trainer import Trainer
import torch
import matplotlib.pyplot as plt

In [5]:
# Load data with class and larget than as targets
dl_train_all, dl_val_all, dl_test_all = load_all_data()

# Load data with class as target
dl_train_class, dl_val_class, dl_test_class = load_class_data()

In [6]:
print("--- Pre-training tail model ---\n")

tail = TailLinear(label_encoded=True)
trainer = Trainer(nb_epochs=5, run='pretrained_tail_tailTraining')
trainer.fit(tail, dl_train_all, dl_val_all)

print("\n--- Training full model ---\n")

tail.requires_grad_=False
le_net = LeNet()
siamese = Siamese(auxiliary=le_net, 
                  target=tail, 
                  softmax=False, 
                  strategy='sum')
trainer = Trainer(nb_epochs=20, run='pretrained_tail_headTraining')
trainer.fit(siamese, dl_train_all, dl_val_all)

print("\n--- Testing model ---\n")
trainer.test(le_net, dl_test_class, return_acc=False)
trainer.test(siamese, dl_test_all, return_acc=False)


--- Pre-training tail model ---

# Epoch 1/5:	 loss=0.66	 loss_val=0.61	 acc_val=84.38
# Epoch 2/5:	 loss=0.51	 loss_val=0.39	 acc_val=96.88
# Epoch 3/5:	 loss=0.27	 loss_val=0.17	 acc_val=99.55
# Epoch 4/5:	 loss=0.12	 loss_val=0.07	 acc_val=100.0
# Epoch 5/5:	 loss=0.05	 loss_val=0.04	 acc_val=100.0

--- Training full model ---



IndexError: Target 8 is out of bounds.

In [4]:
print_param_count(siamese)

Total number of parameters:     83300
Number of trainable parameters: 83300


In [4]:
# Do the same, but 10 times and with verbose=False
results = []
for i in range(10):
    print(f"Run {i+1}/10")

    # Load data with class and larget than as targets
    dl_train_all, dl_val_all, dl_test_all = load_all_data()

    # Training tail model
    tail = TailLinear(label_encoded=True)
    trainer = Trainer(nb_epochs=5, run='pretrained_tail_tailTraining')
    trainer.fit(tail, dl_train_all, dl_val_all, verbose=False)

    # Training full model (but without gradient for last part)
    tail.requires_grad_=False
    le_net = LeNet()
    siamese = Siamese(le_net, tail, softmax=False, strategy='sum')
    trainer = Trainer(nb_epochs=20, run='pretrained_tail_headTraining_argmax')
    trainer.fit(siamese, dl_train_all, dl_val_all, verbose=False)

    # Testing
    acc = trainer.test(siamese, dl_test_all, test_verbose=True, return_acc=True)
    results.extend([acc])

Run 1/10
after one hot encode torch.Size([32, 2, 10])
after flat torch.Size([32, 20])
after one hot encode torch.Size([32, 2, 10])
after flat torch.Size([32, 20])
after one hot encode torch.Size([32, 2, 10])
after flat torch.Size([32, 20])
after one hot encode torch.Size([32, 2, 10])
after flat torch.Size([32, 20])
after one hot encode torch.Size([32, 2, 10])
after flat torch.Size([32, 20])
after one hot encode torch.Size([32, 2, 10])
after flat torch.Size([32, 20])
after one hot encode torch.Size([32, 2, 10])
after flat torch.Size([32, 20])
after one hot encode torch.Size([32, 2, 10])
after flat torch.Size([32, 20])
after one hot encode torch.Size([32, 2, 10])
after flat torch.Size([32, 20])
after one hot encode torch.Size([32, 2, 10])
after flat torch.Size([32, 20])
after one hot encode torch.Size([32, 2, 10])
after flat torch.Size([32, 20])
after one hot encode torch.Size([32, 2, 10])
after flat torch.Size([32, 20])
after one hot encode torch.Size([32, 2, 10])
after flat torch.Size(

after one hot encode torch.Size([32, 2, 10])
after flat torch.Size([32, 20])


IndexError: Target 8 is out of bounds.

In [5]:
# Print results
results = torch.Tensor(results)
print("Average accuracy:   {:.2f}".format(results.mean()))
print("Standard deviation: {:.2f}".format(results.std()))

Average accuracy:   nan
Standard deviation: nan
